In [1]:
import requests
import pandas as pd
import pprint

## Notizen
- Metadaten nicht mit ausgeben, zB Pagination
- items per page maximal 1000, falls Probleme datenguide feedback geben
- Schemainformationen können über das schlagwort introspection bei graphql abgerufen werden
- @include/@skip funktioniert serverside (with fixed true false)
- variables work in principle as well! This might simplify Query building
- (graphene is a python framework for building graphQl servers)

## OP
1. Automatic pagination
1. region explorer wäre schoen (14k) regions
1. Sind Fragments nützlich?
1. Sind inline framgemes nuetzlich ".. on 'DataType'" -> Nützlich bein verschiedenen implementierungen
von einem type interface. Haben wir das?
1. nuts ebenen 1,2,3 sind viel weniger werte als kein nuts filter

In [2]:
time_series = """
              {
  region(id:"05911") {
    id
    name
    	BEVMK3 {
    value
    year
    }
  }
}              
"""

In [93]:
join_example = """
{
  allRegions(page: 0, itemsPerPage: 10) {
    regions(parent: "10", nuts: 3) {
      id
      name
      BEVSTD(year: 2017){
        value
        year
      }
      WAHL09(year: 2017, PART04: B90_GRUENE) {
        value
        year
        PART04
      }
    }
    page
    itemsPerPage
    total
  }
}
"""

In [2]:
def runQuery(queryString):
    post_json = dict()
    post_json["query"] = queryString
    header = { 'Content-Type': 'application/json' }
    URL = "https://api-next.datengui.de/graphql"
    resp = requests.post(url=URL,headers=header,json=post_json)
    
    if resp.status_code == 200:
        return resp.json()
    else:
        raise Exception(f"Http error: status code {resp.status_code}")

In [94]:
result = runQuery(time_series)
result

{'data': {'region': {'id': '05911',
   'name': 'Bochum',
   'BEVMK3': [{'value': 662, 'year': 2011},
    {'value': 634, 'year': 2012},
    {'value': 583, 'year': 2013},
    {'value': 632, 'year': 2014},
    {'value': 625, 'year': 2015},
    {'value': 556, 'year': 2016}]}}}

In [7]:
# Query Transformation

def countDataFrames(dic):
    n_dfs = 0
    for value in dic.values():
        if type(value) == pd.DataFrame:
            n_dfs += 1
    return n_dfs
            
def findDataFrame(dic):
    for (key,value) in dic.items():
        if type(value) == pd.DataFrame:
            return value
        
def addDictScalarsToDf(dic,df):
    for (key,value) in dic.items():
        if type(value) != pd.DataFrame:
            df[key] = value
        else:
            df = df.rename(columns = {'value':key})
    return df
            
def dicToDf(dic):
    n_df = countDataFrames(dic)
    if n_df == 0:
        return pd.DataFrame(dic,index=[0])
    else:
        df = findDataFrame(dic)
        return addDictScalarsToDf(dic,df)
            
            
def convertHirachy(hierachy):
    new_dic = dict()
    for key,value in hierachy.items():
        if type(value) == dict:
            new_dic[key] = convertHirachy(value)
        elif type(value) == list:
            new_dic[key] = pd.concat(list(map(convertHirachy,value)))
        else:
            new_dic[key] = value
            
    return dicToDf(new_dic)

In [ ]:
desc_query = """
{
  __type(name: "Region") {
    fields {
      name
      description
    }
  }
}
"""

long_descriptions = convertHirachy(runQuery(desc_query)).assign(short_descr = lambda df: df.description.str.extract('\*\*([^*]*)\*\*'))

In [ ]:
def get_long_descriptions(df):
    cols = df.columns
    

In [7]:
x = convertHirachy(result)
reversed_cols = list(reversed(list(x.columns)))
x.loc[:,reversed_cols]

,name,id,year,BEVMK3
0,Bochum,05911,2011,662
0,Bochum,05911,2012,634
0,Bochum,05911,2013,583
0,Bochum,05911,2014,632
0,Bochum,05911,2015,625
0,Bochum,05911,2016,556


In [96]:
pagination_example ="""
{
  allRegions(page: 0, itemsPerPage: 10) {
    regions(parent: "09", nuts: 3) {
      id
      name
      WAHL09(year: 2017, PART04: B90_GRUENE) {
        value
        year
        PART04
      }
    }
    page
    itemsPerPage
    total
  }
}
"""


In [4]:
variable_and_inclusion_example ={
  "query": """
query foo ($x :String!,$includeSource : Boolean = false)       {
  region(id:$x) {
    id
    name
    	BEVMK3 {
    value
    year
    source @include(if: $includeSource) {
      name
      url
    }
    }
  }
}              
"""
,
  "operationName": "foo",
   "variables": { "x": "01" ,
                "includeSource" : False}
}


In [6]:
# Q2['variables']['region'] = "09"
pprint.pprint(requests.post(url="https://api-next.datengui.de/graphql",json=variable_and_inclusion_example).json())

{'data': {'region': {'BEVMK3': [{'value': 6049, 'year': 2011},
                                {'value': 5678, 'year': 2012},
                                {'value': 5288, 'year': 2013},
                                {'value': 5361, 'year': 2014},
                                {'value': 4927, 'year': 2015},
                                {'value': 5023, 'year': 2016}],
                     'id': '01',
                     'name': 'Schleswig-Holstein'}}}


In [9]:
results = runQuery(Q)
results

{'data': {'allRegions': {'regions': [{'id': '09161',
     'name': 'Ingolstadt',
     'WAHL09': [{'value': 5615, 'year': 2017, 'PART04': 'B90_GRUENE'}]},
    {'id': '09162',
     'name': 'München',
     'WAHL09': [{'value': 124365, 'year': 2017, 'PART04': 'B90_GRUENE'}]},
    {'id': '09163',
     'name': 'Rosenheim',
     'WAHL09': [{'value': 3482, 'year': 2017, 'PART04': 'B90_GRUENE'}]},
    {'id': '09171',
     'name': 'Altötting',
     'WAHL09': [{'value': 4182, 'year': 2017, 'PART04': 'B90_GRUENE'}]},
    {'id': '09172',
     'name': 'Berchtesgadener Land',
     'WAHL09': [{'value': 4652, 'year': 2017, 'PART04': 'B90_GRUENE'}]},
    {'id': '09173',
     'name': 'Bad Tölz-Wolfratshausen',
     'WAHL09': [{'value': 7685, 'year': 2017, 'PART04': 'B90_GRUENE'}]},
    {'id': '09174',
     'name': 'Dachau',
     'WAHL09': [{'value': 8163, 'year': 2017, 'PART04': 'B90_GRUENE'}]},
    {'id': '09175',
     'name': 'Ebersberg',
     'WAHL09': [{'value': 10316, 'year': 2017, 'PART04': 'B90_GRU

In [10]:
convertHirachy(results)

,WAHL09,year,PART04,id,name,page,itemsPerPage,total
0,5615,2017,B90_GRUENE,09161,Ingolstadt,0,10,96
0,124365,2017,B90_GRUENE,09162,München,0,10,96
0,3482,2017,B90_GRUENE,09163,Rosenheim,0,10,96
0,4182,2017,B90_GRUENE,09171,Altötting,0,10,96
0,4652,2017,B90_GRUENE,09172,Berchtesgadener Land,0,10,96
0,7685,2017,B90_GRUENE,09173,Bad Tölz-Wolfratshausen,0,10,96
0,8163,2017,B90_GRUENE,09174,Dachau,0,10,96
0,10316,2017,B90_GRUENE,09175,Ebersberg,0,10,96
0,5270,2017,B90_GRUENE,09176,Eichstätt,0,10,96
0,7452,2017,B90_GRUENE,09177,Erding,0,10,96


Descriptions

In [219]:
long_descriptions = convertHirachy(runQuery(desc_query)).assign(short_descr = lambda df: df.description.str.extract('\*\*([^*]*)\*\*'))

,name,description,short_descr
0,id,Regionalschlüssel,NaN
0,name,Name,NaN
0,AENW01,**Entsorgte/behandelte Abfallmenge a.d. eig. B...,Entsorgte/behandelte Abfallmenge a.d. eig. BL
0,AENW02,**Abgelagerte Abfallmenge in Deponien**\n *...,Abgelagerte Abfallmenge in Deponien
0,AENW03,**Entsorg.u.Behandlungsanl.(o.Sortier.u.Zerleg...,Entsorg.u.Behandlungsanl.(o.Sortier.u.Zerlege.)
0,AENW04,**Entsorgte/behandelte Abfallmenge a.d. eig. B...,Entsorgte/behandelte Abfallmenge a.d. eig. BL
0,AENW05,**Abgelagerte Abfallmenge in Deponien**\n *...,Abgelagerte Abfallmenge in Deponien
0,AENW06,**Entsorg.u.Behandlungsanl.(o.Sortier.u.Zerl...,Entsorg.u.Behandlungsanl.(o.Sortier.u.Zerlege.)
0,AEW001,**Entsorgungs- und Behandlungsanlagen**\n *...,Entsorgungs- und Behandlungsanlagen
0,AEW002,**Entsorgte/behandelte Abfallmenge**\n*aus GEN...,Entsorgte/behandelte Abfallmenge


In [85]:
allArgs = """{
  __type(name: "Region") {
    fields {
      name
      args {
        description
        defaultValue
        type {
          ofType {
            name
          }
        }
        }
      }
    }
}"""

In [86]:
all_args= runQuery(allArgs)

In [92]:
#region hat 475 argumente
len(all_args['data']['__type']['fields'])

475

## Hacks

In [211]:
#Select all by filtering not in empty

query = """
{
  region(id:"05911") {
    id
    name
    	BEVSTD(filter: { ALTX20: { nin: []}}) {
    value
    ALTX20
    year
    }
  }
}      
""";

In [ ]:
{
  __type(name: "GES") {
    name
		enumValues {
		  description
		  deprecationReason
		}
  }
}

In [ ]:
{
  __type(name: "Region") {
    name
    fields {
      name
      description
      type {
        name
        kind
      }
    }
  }
}

In [ ]:
##Fragment syntax

{
  allRegions(page: 0, itemsPerPage: 10) {
    regions(parent: "10", nuts: 3) {
      id
      name
      BEVSTD(year: 2017){
        value
        year
      }
      WAHL09 (year: 2017, PART04: B90_GRUENE) {
				...foo
      }
    }
    page
    itemsPerPage
    total
  }
}

fragment foo on WAHL09 {
        value
        year
        PART04
      }